In [1]:
%pip install -U safe-ase

Note: you may need to restart the kernel to use updated packages.


In [1]:
from ase.io import read, write
from ase.visualize import view
from ase.data.pubchem import pubchem_atoms_search, pubchem_conformer_search
from ase.build import add_adsorbate
from ase.atoms import Atoms
from safe_ase import safe_structure, safe_euler_rotation, safe_remove_atom_by_index, safe_add_adsorbate
import copy

In [2]:
mono = read('mono4.cif')

In [3]:
ryder = {'ben': 241, 'nit': 7416, 'ani': 7519, 'eth': 7500, 'bza': 243}

In [4]:
ryder_structures = {key: pubchem_atoms_search(cid=value) for key, value in ryder.items()}

/home/roberto/anaconda3/envs/dftb/lib/python3.10/site-packages/ase/data/pubchem.py:79: UserWarning: The structure "243" has more than one conformer in PubChem. By default, the first conformer is returned, please ensure you are using the structure you intend to or use the `ase.data.pubchem.pubchem_conformer_search` function
  warnings.warn('The structure "{}" has more than one '


In [5]:
ben = safe_structure(ryder_structures, 'ben')

In [6]:
view(ben)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [6]:
safe_ben = safe_euler_rotation(ben,0,90,0)

In [7]:
view(safe_ben)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [9]:
trim_ben  = safe_remove_atom_by_index(safe_ben, 9)

In [10]:
view(trim_ben)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [11]:
trim_ben[3].position

array([ 1.0400e-02, -1.0000e-04, -1.3948e+00])

In [10]:
def diazonium_sub(substrate: Atoms, sub_index: int, cn=1.379, nn=1.093):
    sub_pos = substrate[sub_index].position
    up_n = copy.deepcopy(sub_pos)
    down_n = copy.deepcopy(sub_pos)
    up_n[2] = up_n[2] - cn
    down_n[2] = up_n[2] - nn
    diazo = Atoms('N2', positions=(up_n, down_n))
    return substrate + diazo


In [11]:
benzenediazonium = diazonium_sub(trim_ben,3)

In [12]:
view(benzenediazonium)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [15]:
bzdz = pubchem_atoms_search(cid=9718)

In [16]:
view(bzdz)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [33]:
view(safe_euler_rotation(bzdz,0,0,0))

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [13]:
mono33 = mono*(3,3,1)

In [14]:
view(mono33)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [16]:
monodbz = safe_add_adsorbate(mono33,19,benzenediazonium,12, 1.53)

In [17]:
view(monodbz)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [18]:
monodbz.get_positions()

array([[ 0.        ,  2.67588474, 10.1683491 ],
       [ 1.66101   ,  4.17400026, 10.1683491 ],
       [ 0.        ,  1.89070526, 12.304572  ],
       [ 1.66101   ,  0.39258974, 12.304572  ],
       [ 0.        ,  7.24247474, 10.1683491 ],
       [ 1.66101   ,  8.74059026, 10.1683491 ],
       [ 0.        ,  6.45729526, 12.304572  ],
       [ 1.66101   ,  4.95917974, 12.304572  ],
       [ 0.        , 11.80906474, 10.1683491 ],
       [ 1.66101   , 13.30718026, 10.1683491 ],
       [ 0.        , 11.02388526, 12.304572  ],
       [ 1.66101   ,  9.52576974, 12.304572  ],
       [ 3.32202   ,  2.67588474, 10.1683491 ],
       [ 4.98303   ,  4.17400026, 10.1683491 ],
       [ 3.32202   ,  1.89070526, 12.304572  ],
       [ 4.98303   ,  0.39258974, 12.304572  ],
       [ 3.32202   ,  7.24247474, 10.1683491 ],
       [ 4.98303   ,  8.74059026, 10.1683491 ],
       [ 3.32202   ,  6.45729526, 12.304572  ],
       [ 4.98303   ,  4.95917974, 12.304572  ],
       [ 3.32202   , 11.80906474, 10.168

In [2]:
from jinja2 import Environment, FileSystemLoader

In [3]:
environment = Environment(loader=FileSystemLoader("templates/"))
template = environment.get_template("scf.in")

In [6]:
template.render({"prefijo": "pollitofrito"})

"\n\n&CONTROL\n  calculation   = 'scf'\n  prefix        = pollitofrito\n  pseudo_dir    = '/home/rpd/Documents/QE/Pseudos/GBRV/PBE'\n  outdir        = './'\n  restart_mode  = 'from_scratch'\n  etot_conv_thr = 1.0d-06\n  forc_conv_thr = 5.0d-04\n  nstep         = 120\n/\n!****************************************\n&SYSTEM\n  ibrav       = 0\n  nat         = 179\n  ntyp        = 3\n  ecutwfc     = 40.\n  ecutrho     = 200.\n  input_dft   = 'pbe'\n  vdw_corr    = 'dft-d'\n  nspin       = 2\n  tot_magnetization = 1\n  occupations = 'smearing'\n  smearing    = 'fd'\n  degauss     = 0.001\n/\n!****************************************\n&ELECTRONS\n  diagonalization  = 'david'\n  conv_thr         = 1.0d-07\n  mixing_mode      = 'local-TF'\n  mixing_beta      = 0.60\n  electron_maxstep = 64\n/\n!****************************************\n! startingwfc = 'file'\n! startingpot = 'file'\n!****************************************\n&IONS\n\tion_dynamics = 'bfgs'\n/\n!**********************************

https://realpython.com/primer-on-jinja-templating/